# Create CSV file for Review data


In [1]:
import json
from amazon_scraper import AmazonScraper

scraper = AmazonScraper()
asin = "B09C6HGX1W"  # Example ASIN

product = scraper.scrape_product_reviews(asin)

Scraping reviews sorted by: recent
Scraping page 1 for sort by recent and star rating all_star and format type all_formats
URL: https://www.amazon.com/product-reviews/B09C6HGX1W?sortBy=recent&pageNumber=1&filterByStar=all_star&formatType=all_formats&mediaType=media_reviews_only
Page 1 scraped successfully
HTML length: 478114
Found 10 review elements
Scraping reviews sorted by: recent
Scraping page 1 for sort by recent and star rating all_star and format type all_formats
URL: https://www.amazon.com/product-reviews/B09C6HGX1W?sortBy=recent&pageNumber=1&filterByStar=all_star&formatType=all_formats&mediaType=all_contents
Page 1 scraped successfully
HTML length: 424828
Found 10 review elements
Scraping reviews sorted by: recent
Scraping page 1 for sort by recent and star rating all_star and format type current_format
URL: https://www.amazon.com/product-reviews/B09C6HGX1W?sortBy=recent&pageNumber=1&filterByStar=all_star&formatType=current_format&mediaType=media_reviews_only
Page 1 scraped su

In [2]:
# Serializing json
json_object = json.dumps(product.to_dict(), indent=4)

# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [3]:
import pandas as pd

# Load the JSON file into a pandas DataFrame
with open("sample.json", "r") as file:
    data = json.load(file)  # Load the JSON data into a dictionary

# Assuming the structure contains 'review_list', we normalize this list into a DataFrame
reviews_df = pd.json_normalize(data, "review_list", errors="ignore")

# Display the first few rows of the DataFrame to check
print(reviews_df.head())

# Count distinct review IDs
distinct_review_count = reviews_df["id"].nunique()

print(f"Number of distinct review IDs: {distinct_review_count}")

               id  rating                                              title  \
0  R1JPN9L2129WFQ     1.0                 Great but stoped working after 1yr   
1  R1XKMO29V1XWUX     2.0  It’s excellent in design and materials on exte...   
2   RP4MFOK7UP280     1.0                        Received a damaged and USED   
3  R3M8MDZNUHA46E     5.0  true hot towel like shave! Best shave ever, an...   
4  R39E02DDHJCRS8     4.0                                               eciN   

                                                href            country  \
0  /gp/customer-reviews/R1JPN9L2129WFQ/ref=cm_cr_...  the United States   
1  /gp/customer-reviews/R1XKMO29V1XWUX/ref=cm_cr_...  the United States   
2  /gp/customer-reviews/RP4MFOK7UP280/ref=cm_cr_a...  the United States   
3  /gp/customer-reviews/R3M8MDZNUHA46E/ref=cm_cr_...  the United States   
4  /gp/customer-reviews/R39E02DDHJCRS8/ref=cm_cr_...  the United States   

                  date                                              